In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

import os, sys


In [115]:
cwd = os.getcwd()
train_data_path = os.path.join(cwd, 'data', 'train.csv')
test_data_path = os.path.join(cwd, 'data', 'test.csv')
sample_submission_path = os.path.join(cwd, 'data', 'sample_submission.csv')

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
sample_submission = pd.read_csv(sample_submission_path)

In [116]:
train_data.head()

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_17,x_18,x_19,x_20,x_21,x_22,x_23,x_24,x_25,labels
0,-0.789364,-1.434296,0.324867,-1.089570,-3.186751,-1.915492,-1.985052,-5.109614,-1.776943,-10.228432,...,-1.145350,-1.157258,-4.935825,1.567903,1.691138,-2.914742,0.713525,1.703421,-0.222027,0.0
1,1.698561,-0.530175,0.756504,-0.849795,0.738859,4.321680,3.322877,-4.769473,-1.148654,-0.623213,...,1.077191,-3.360013,-7.324134,1.074675,-0.984185,-1.361525,2.444832,-1.497029,1.096920,0.0
2,2.480805,-2.933747,1.407295,-0.356059,1.179147,-4.181063,-4.177118,-2.854105,-0.223620,-2.034928,...,-3.219309,-0.026445,-8.659095,-0.152213,0.685907,0.442014,1.818607,2.793273,0.072918,2.0
3,1.457755,-0.106902,-0.852411,1.175998,6.619029,2.156072,-0.777952,5.031490,0.476906,3.009128,...,4.263037,-0.784668,5.115430,1.010681,-0.641215,-6.322318,-0.806044,0.699570,0.260674,2.0
4,1.427555,0.649282,0.254497,-1.064585,0.303576,-4.545240,4.577316,-5.233015,-1.007334,0.511030,...,3.363232,-1.767302,-3.108375,-1.916984,-0.423236,0.631079,-3.378547,-4.165684,0.967436,2.0


In [117]:
test_data.head()

,ID,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_16,x_17,x_18,x_19,x_20,x_21,x_22,x_23,x_24,x_25
0,1,-0.230293,-3.466028,1.511166,0.740295,3.696918,-2.578689,2.263205,-0.126368,1.207075,...,-0.717038,-2.280132,-4.019121,4.175089,-2.939001,0.690836,-1.537785,0.523352,-0.287075,-0.033105
1,2,-0.589310,2.695952,-0.447133,1.742419,-3.912262,7.050236,-2.624268,2.292610,-0.640342,...,-3.313892,-7.084135,-0.161589,-3.913306,-3.592095,-2.974472,2.576795,-1.702104,2.209905,0.618079
2,3,2.070704,-1.921016,1.352349,1.948624,-1.549088,-0.623295,-0.013214,4.281549,-0.569961,...,1.295106,0.363587,2.207610,4.304411,-1.301508,-3.051108,1.138168,-2.822654,-1.628571,-0.441178
3,4,0.130017,2.225700,-0.504748,-0.401777,2.244243,4.770526,1.789050,-1.553924,1.116070,...,8.556711,-5.356854,2.574727,6.959246,0.220325,0.344151,3.047017,1.398412,-0.284969,-0.424696
4,5,-1.718615,0.253217,-0.539986,0.261817,0.246253,-0.502865,-1.190270,-1.416252,-1.735776,...,-4.632323,-1.219645,2.092873,-2.675771,-4.998719,1.639839,-1.230900,2.967112,0.752419,-0.589382


In [118]:
X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

X_test = test_data.iloc[:,1:]
X_test.head()



,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_16,x_17,x_18,x_19,x_20,x_21,x_22,x_23,x_24,x_25
0,-0.230293,-3.466028,1.511166,0.740295,3.696918,-2.578689,2.263205,-0.126368,1.207075,3.171999,...,-0.717038,-2.280132,-4.019121,4.175089,-2.939001,0.690836,-1.537785,0.523352,-0.287075,-0.033105
1,-0.589310,2.695952,-0.447133,1.742419,-3.912262,7.050236,-2.624268,2.292610,-0.640342,-15.378380,...,-3.313892,-7.084135,-0.161589,-3.913306,-3.592095,-2.974472,2.576795,-1.702104,2.209905,0.618079
2,2.070704,-1.921016,1.352349,1.948624,-1.549088,-0.623295,-0.013214,4.281549,-0.569961,1.071801,...,1.295106,0.363587,2.207610,4.304411,-1.301508,-3.051108,1.138168,-2.822654,-1.628571,-0.441178
3,0.130017,2.225700,-0.504748,-0.401777,2.244243,4.770526,1.789050,-1.553924,1.116070,-5.739190,...,8.556711,-5.356854,2.574727,6.959246,0.220325,0.344151,3.047017,1.398412,-0.284969,-0.424696
4,-1.718615,0.253217,-0.539986,0.261817,0.246253,-0.502865,-1.190270,-1.416252,-1.735776,-7.300250,...,-4.632323,-1.219645,2.092873,-2.675771,-4.998719,1.639839,-1.230900,2.967112,0.752419,-0.589382


In [119]:
# Define a function to train the logistic regression model
def train_model(X_train, y_train, C):
  # Create a logistic regression model with L1 regularization
  clf = GaussianNB()

  # Fit the model to the training data
  clf.fit(X_train, y_train)

  return clf

# Tune the regularization strength using cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_score = 0
best_C = 0
for C in np.arange(0.01, 10.0, 0.1):
  scores = []
  for train_index, test_index in kf.split(X_train_scaled):
    X_train_fold, X_test_fold = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

    # Train the model on the training fold
    clf = train_model(X_train_fold, y_train_fold, C)

    # Make predictions on the test fold
    y_pred = clf.predict(X_test_fold)

    # Calculate the accuracy score
    accuracy = np.mean(y_pred == y_test_fold)

    scores.append(accuracy)

  # Calculate the average accuracy score across the folds
  average_score = np.mean(scores)

  if average_score > best_score:
    best_score = average_score
    best_C = C

# Train the model on the entire training data using the best regularization strength
clf = train_model(X_train_scaled, y_train, best_C)

# Standardize the test features
X_test_scaled = scaler.transform(X_test)

# Make predictions on the test data
y_pred = clf.predict(X_test_scaled)

# Update the predicted labels in the sample submission file
sample_submission['label'] = y_pred

# Save the modified sample submission file
sample_submission.to_csv('modified_sample_submission.csv', index=False)